**Downloading Aerial Images using Google Maps Static API**

In [5]:
import requests
import pandas as pd
from pathlib import Path
import geopandas as gpd
from tqdm import tqdm

In [2]:

# Load the data
data_path = Path('/Users/wolff/Desktop/delft_download/data/Rotterdam_NL')
panoids = gpd.read_file(data_path / 'panoids/panoids.geojson')  # Assuming you have a CSV version of the file
panoid_copy = panoids.copy()

In [16]:
df = pd.read_csv('funda_all_cleaned_may_23.csv')
#df = df.sample(10)

In [17]:
df.shape

(6842, 26)

In [19]:
# Function to get aerial image from Google Maps Static API
def get_aerial_image(lat, lng, zoom, api_key, save_path):
    url = "https://maps.googleapis.com/maps/api/staticmap"
    params = {
        'center': f'{lat},{lng}',
        'zoom': zoom,
        'size': '600x600',
        'maptype': 'satellite',
        'key': api_key
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        with open(save_path, 'wb') as file:
            file.write(response.content)
        return save_path
    else:
        print(f"Failed to retrieve image for {lat}, {lng}. Status code: {response.status_code}")
        return None

# Define your Google Maps API key
api_key = 'xxx'

# Specify the zoom level
zoom_level = 18  # Adjust this zoom level as needed

# Directory to save images
image_save_path = data_path / 'aerial_images'
image_save_path.mkdir(exist_ok=True)

# List to store data for the new dataframe
data = []

# Fetch and save aerial images for each entry in df with a progress bar
for idx, row in tqdm(df.iterrows(), total=len(df), desc="Fetching images"):
    lat = row['latitude']
    lng = row['longitude']
    full_address = row['full_address']
    save_path = image_save_path / f'aerial_image_{idx}.png'
    image_path = get_aerial_image(lat, lng, zoom_level, api_key, save_path)
    if image_path:
        data.append({
            'full_address': full_address,
            'latitude': lat,
            'longitude': lng,
            'image_path': str(image_path),
        })

# Create the new dataframe
new_df = pd.DataFrame(data)

# Save the new dataframe to a CSV file
new_df.to_csv(data_path / 'aerial_images_metadata1.csv', index=False)

print("New dataframe saved to aerial_images_metadata.csv")

Fetching images: 100%|██████████| 6842/6842 [25:52<00:00,  4.41it/s]

New dataframe saved to aerial_images_metadata.csv


In [20]:
# Display the first few rows of both dataframes to compare
print("Input DataFrame (df):")
print(df[['full_address', 'latitude', 'longitude']].head())

print("\nOutput DataFrame (new_df):")
print(new_df.head())

# Merge both dataframes to facilitate comparison
comparison_df = df[['full_address', 'latitude', 'longitude']].merge(
    new_df, on=['full_address', 'latitude', 'longitude'], how='left'
)

# Display the merged dataframe to check the comparison
print("\nComparison DataFrame:")
print(comparison_df.head())
print(comparison_df.shape)
print(df.shape)
print(new_df.shape)

Input DataFrame (df):
                                    full_address   latitude  longitude
0  Menno ter Braakstraat 120, 3076 EJ, rotterdam  51.875350   4.518130
1    Schiebroeksestraat 12 A, 3037 RV, rotterdam  51.938222   4.475345
2    Graaf Balderikstraat 44, 3032 HC, rotterdam  51.926685   4.482866
3          Beukelsdijk 110 B, 3022 DK, rotterdam  51.920571   4.452607
4               Westplein 89, 3016 BM, rotterdam  51.909052   4.478779

Output DataFrame (new_df):
                                    full_address   latitude  longitude  \
0  Menno ter Braakstraat 120, 3076 EJ, rotterdam  51.875350   4.518130   
1    Schiebroeksestraat 12 A, 3037 RV, rotterdam  51.938222   4.475345   
2    Graaf Balderikstraat 44, 3032 HC, rotterdam  51.926685   4.482866   
3          Beukelsdijk 110 B, 3022 DK, rotterdam  51.920571   4.452607   
4               Westplein 89, 3016 BM, rotterdam  51.909052   4.478779   

                                          image_path  
0  /Users/wolff/Desktop/